In [1]:
import importlib

import os
import sys
import json
import warnings
import traceback

#from scipy import signal
import numpy as np
from tqdm import tqdm

sys.path.insert(0, '/mnt/d/projects/bassline_extraction') 

from plotting.various import plot_energy_levels
import bassline_transcription as transcription

from signal_processing import *
from chorus_estimation import *
from utilities import *

warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)

In [2]:
with open('../data/ouz_tracks.txt', 'r') as infile:
    track_titles = infile.read().split('\n')
       
_,_, bad_examples = next(os.walk('../data/bassline_extraction/beat_grid/bad_examples'))
bad_examples = [title.split('.txt')[0] for title in bad_examples]
       
with open('../data/metadata/scales_frequencies.json','r') as infile:
    scales = json.load(infile)

with open('../data/metadata/TechHouse_track_dicts.json','r') as infile:
    track_dicts = json.load(infile) 

In [3]:
title = track_titles[np.random.randint(len(track_titles))] 
print('{}'.format(title))
BPM = float(track_dicts[title]['BPM'])
print('BPM: {}\n'.format(BPM))
beat_length = 60/BPM

fs = 44100

if title in bad_examples:
    with open(os.path.join('../data/bassline_extraction/beat_grid/bad_examples',title+'.txt'), 'r') as infile:
        dpqb = float(infile.read())
    print("Beat grid problematic! dpqb: {:.2f}%\n{}".format(dpqb,title))
    

beat_positions = get_beat_positions(title)

# estimated beat lengths from the beat positions
beat_length_estimations = np.diff(beat_positions)

ATFC & David Penn - Hipcats (Extended Mix)
BPM: 122.0



In [4]:
beat_factor = 64

tolerance = beat_length/beat_factor

print('Beat Length:           {:.5f}'.format(beat_length))
print('Tolerance:             {:.5f}'.format(tolerance))
print('Possible lengths: {:.5f} - {:.5f}'.format(beat_length-tolerance, beat_length+tolerance))

# how much each beat length is deviated from the expected value
length_deviations = np.abs(beat_length - beat_length_estimations)

# where does the length variations occur
large_deviation_indices = np.where(length_deviations>tolerance)[0]
print('\nThere are {} beats with large length deviation.'.format(len(large_deviation_indices)))
print('\nReal Indices: \n{}'.format(large_deviation_indices))
print('Corresponding beat lengths: \n{}'.format(beat_length_estimations[large_deviation_indices]))

# find where the large variations are continuous
continuous_regions = calcRegionBounds(np.diff(large_deviation_indices) <= 4)
# end idx included in the bad region

# get the corresponding indices in the beat_length_estimations
bad_regions = []
for start, end in continuous_regions: # end idx is included in the bad region
    
    real_start_idx = large_deviation_indices[start]     
    real_end_idx = large_deviation_indices[end]
    
    bad_regions.append([real_start_idx, real_end_idx])

print('\nBeat lengths inside the bad regions')
for start, end in bad_regions:
    print(beat_length_estimations[start:end+1])

Beat Length:           0.49180
Tolerance:             0.00768
Possible lengths: 0.48412 - 0.49949

There are 152 beats with large length deviation.

Real Indices: 
[  0   1   7  13  17  23  29  35  41  45  51  57  63  67  73  79  85  91
  97 101 107 113 119 123 129 135 141 145 151 157 163 167 168 169 173 179
 185 189 190 191 195 201 207 211 217 223 229 235 239 240 241 247 251 257
 262 267 273 279 280 281 282 284 285 286 288 289 295 299 307 311 317 323
 327 335 339 345 351 354 360 366 371 377 383 388 394 399 405 411 417 422
 427 433 439 444 452 457 463 468 473 474 476 478 483 489 495 499 507 511
 517 523 527 533 539 545 551 555 561 567 571 577 583 589 595 599 605 611
 617 623 629 633 639 645 651 655 661 667 673 677 678 679 683 689 695 699
 705 711 717 723 727 733 735 736]
Corresponding beat lengths: 
[0.48 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.5  0.5  0.5  0.5  0.48 0.5  0.5  0.5  0.5  0.5

In [5]:
# create te good region bounds
# end idx is included in the good region
good_regions = [[0, bad_regions[0][0]-1]]
for i, (_, bad_end_idx) in enumerate(bad_regions[:-1]):
    
    s = bad_end_idx+1
    e = bad_regions[i+1][0]-1
    
    if s != e:
        good_regions.append([s, e])
        
s = bad_regions[i+1][1]+1
e = len(beat_length_estimations)

if s != e:
    good_regions.append([s, e])

In [6]:
patches = []
for start, end in bad_regions:
    
    t0, t1 = beat_positions[start-1], beat_positions[end+1]
    
    no_beats = np.around((t1 - t0)/beat_length).astype(int)
    
    #print('No beats: {}'.format(no_beats))
    
    patch = t0 + np.arange(no_beats)*beat_length
    
    patches.append(patch)
    
    #print(t0, t1)
    #print(patch)
    #print(patch[-1]-t1)
    #print('\n')

In [7]:
reconstructed_beat_positions = []
for i, (start, end) in enumerate(good_regions):
    
    reconstructed_beat_positions.extend(beat_positions[start:end+1])
    
    reconstructed_beat_positions.extend(patches[i])

In [8]:
reconstructed_beat_positions

[1.03,
 1.52,
 2.01,
 2.5,
 2.99,
 3.48,
 3.98,
 4.47,
 4.96,
 5.45,
 5.94,
 5.94,
 6.431803278688525,
 6.92360655737705,
 7.4154098360655745,
 7.907213114754098,
 8.399016393442624,
 8.9,
 9.39,
 9.88,
 10.37,
 10.86,
 11.35,
 11.85,
 12.34,
 12.83,
 13.32,
 13.81,
 14.3,
 14.8,
 15.29,
 15.78,
 16.27,
 16.76,
 17.25,
 17.75,
 18.24,
 18.73,
 19.22,
 19.71,
 19.71,
 20.201803278688526,
 20.69360655737705,
 21.185409836065574,
 21.6772131147541,
 22.169016393442625,
 22.67,
 23.16,
 23.65,
 24.14,
 24.63,
 25.12,
 25.62,
 26.11,
 26.6,
 27.09,
 27.58,
 28.07,
 28.57,
 29.06,
 29.55,
 30.04,
 30.53,
 30.53,
 31.021803278688527,
 31.51360655737705,
 32.005409836065574,
 32.497213114754096,
 32.989016393442625,
 33.49,
 33.98,
 34.47,
 34.96,
 35.45,
 35.94,
 36.44,
 36.93,
 37.42,
 37.91,
 38.4,
 38.89,
 39.39,
 39.88,
 40.37,
 40.86,
 41.35,
 41.84,
 42.34,
 42.83,
 43.32,
 43.81,
 44.3,
 44.79,
 45.29,
 45.78,
 46.27,
 46.76,
 47.25,
 47.25,
 47.74180327868852,
 48.23360655737705,
 48.